# Outcome Prediction with RGCN Model

## 1. Load data from "processed_data" folder

In [17]:
import time
import math
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from torch_geometric.utils import to_undirected

In [18]:
num_patients = 1000
inverse_triples=True
embed_dim = 100

entity = pd.read_csv(f'processed_data/sphn_entities_{num_patients}_noOutcome.tsv', sep='\t', index_col=0, header=None)
entity = entity.to_dict()[1]

indices = []
for i in range(num_patients):
    idx = f'<http://nvasc.org/synth_patient_{i}>'
    indices.append(entity[idx])

events = pd.read_csv(f'processed_data/sphn_events_{num_patients}_noOutcome.tsv', sep='\t', header=None)
y = joblib.load(f'../Data Generation/outcomes_{num_patients}_0.joblib')

non_test_X, test_X, non_test_y, test_y_ = train_test_split(indices, y, stratify=y, test_size=0.2)
train_X, valid_X, train_y_, valid_y_ = train_test_split(non_test_X, non_test_y, stratify=non_test_y, test_size=1./8)

edge_index = torch.vstack((torch.Tensor(events[0]).long(),torch.Tensor(events[2]).long()))
edge_type = torch.Tensor(events[1]).long()
train_idx = torch.Tensor(train_X).long()
train_y = torch.Tensor(train_y_).long()
valid_idx = torch.Tensor(valid_X).long()
valid_y = torch.Tensor(valid_y_).long()
test_idx = torch.Tensor(test_X).long()
test_y = torch.Tensor(test_y_).long()
num_nodes = len(entity)

if inverse_triples == True:
    edge_index = to_undirected(edge_index)
    edge_type = torch.cat((edge_type, edge_type))

data = Data(
    edge_index=edge_index,
    edge_type=edge_type,
    train_idx=train_idx,
    train_y=train_y,
    valid_idx=valid_idx,
    valid_y=valid_y,
    test_idx=test_idx,
    test_y=test_y,
    num_nodes=num_nodes,
)
embedding = torch.nn.Parameter(torch.empty(num_nodes, embed_dim))
torch.nn.init.xavier_uniform_(embedding, gain=math.sqrt(2.0))
data.x = embedding
data.num_relations = data.num_edge_types
data.num_classes = 3
print(data)

Data(edge_index=[2, 221870], edge_type=[221870], train_idx=[700], train_y=[700], valid_idx=[100], valid_y=[100], test_idx=[200], test_y=[200], num_nodes=33563, x=[33563, 100], num_relations=10, num_classes=3)


## 2. Training the model

In [19]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = RGCNConv(embed_dim, 16, data.num_relations,
                          num_bases=4)
        self.conv2 = RGCNConv(16, data.num_classes, data.num_relations,
                          num_bases=4)

    def forward(self, edge_index, edge_type):
        x = F.elu(self.conv1(data.x, edge_index, edge_type))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    val_acc = float((pred[data.valid_idx] == data.valid_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, val_acc, test_acc

In [21]:
times = []
for epoch in range(1, 501):
    start = time.time()
    loss = train()
    if epoch % 10 == 0:
        train_acc, val_acc, test_acc = test()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f} | Train: {train_acc:.4f} '
            f'Val: {val_acc:.4f} Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 10, Loss: 0.9490 | Train: 0.6171 Val: 0.4500 Test: 0.4700
Epoch: 20, Loss: 0.8824 | Train: 0.7057 Val: 0.5200 Test: 0.5100
Epoch: 30, Loss: 0.7701 | Train: 0.7700 Val: 0.4900 Test: 0.4700
Epoch: 40, Loss: 0.6273 | Train: 0.8271 Val: 0.4700 Test: 0.4900
Epoch: 50, Loss: 0.4557 | Train: 0.8929 Val: 0.4900 Test: 0.5000
Epoch: 60, Loss: 0.3245 | Train: 0.9614 Val: 0.4400 Test: 0.5000
Epoch: 70, Loss: 0.2571 | Train: 0.9900 Val: 0.4500 Test: 0.4950
Epoch: 80, Loss: 0.1782 | Train: 0.9957 Val: 0.4400 Test: 0.4700
Epoch: 90, Loss: 0.1584 | Train: 0.9971 Val: 0.4500 Test: 0.4700
Epoch: 100, Loss: 0.1272 | Train: 0.9986 Val: 0.4300 Test: 0.4950
Epoch: 110, Loss: 0.1286 | Train: 0.9986 Val: 0.4300 Test: 0.5000
Epoch: 120, Loss: 0.1200 | Train: 1.0000 Val: 0.4100 Test: 0.4900
Epoch: 130, Loss: 0.1124 | Train: 1.0000 Val: 0.4300 Test: 0.4900
Epoch: 140, Loss: 0.0945 | Train: 1.0000 Val: 0.4100 Test: 0.5050
Epoch: 150, Loss: 0.0974 | Train: 1.0000 Val: 0.4400 Test: 0.5050
Epoch: 160, Loss: 0

In [22]:
from sklearn.metrics import roc_auc_score

model.eval()
with torch.no_grad():
    out = model(data.edge_index, data.edge_type).cpu()
    prob = F.softmax(out, dim=1)
auc = roc_auc_score(data.test_y.cpu(), prob[data.test_idx.cpu()], multi_class='ovr')
print(f'ROC AUC score: {auc:.4f}')

ROC AUC score: 0.6170
